<a href="https://colab.research.google.com/github/aya-rhouma/C-Project/blob/main/RAGaiNightChallenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pypdf2 pdfplumber tqdm

In [ ]:
!pip install gdown

import gdown
import os

folder_id = "1coBEkgPanCSTXkud4SjbKbZvW_P2p5fw"

!gdown --folder https://drive.google.com/drive/folders/{folder_id}

pdf_files = [f for f in os.listdir('.') if f.endswith('.pdf')]
print(f"{len(pdf_files)} fichiers PDF trouvés :")
for f in pdf_files[:5]:
    print(f"  - {f}")

Retrieving folder contents
Processing file 1rLtLYfaeMyP6nnhb5BOYeX5IkJvqMpyf acide ascorbique.pdf
Processing file 1Lmaz8GvZHdnNnmXL1cgZ6ozit1zSt55f BVZyme  GOX 110 TDS(1).pdf
Processing file 14yk3RzNJCnHKmKCvEOxX1dfZnzBcTDno BVZyme  TG MAX63 TDS.pdf
Processing file 1AaidmJrSQgrBZoDXvx2h65cVhYLqOxzo BVZyme  TG MAX64 TDS.pdf
Processing file 1xZztuSrSbZyrlynujSAxQE5mhwCaaGQ4 BVZyme  TG881 TDS(1).pdf
Processing file 1YFsbDLzJqnJfPpWMN74SqhoMeAXDCqU0 BVZyme  TG883 TDS.pdf
Processing file 18aWu2Z7dW6swEA8Nmc0WXFkav6d-NlNz BVZyme GO MAX 63 TDS.pdf
Processing file 1Qo6K7zIilaCKob6FHDkp4iTBaGeCjsDx BVZyme GO MAX 65.pdf
Processing file 1nJVJKXHgUKoanJtrT0fYZFr30g91RXK9 BVZyme TDS A FRESH101.pdf
Processing file 1Up1N5rNXc8tokzlbD7Sg_uOIZSMPQlX7 BVZyme TDS A FRESH202.pdf
Processing file 1p1QLcqF9tr9R6ZjprtvN7lz4BYb__yhL BVZyme TDS A FRESH303.pdf
Processing file 1BKwtjufbuGM_-5Ox8KooNs3QsO1L4kP9 BVZyme TDS A SOFT205.pdf
Processing file 1T-hgHInp-6NNPD2rJ3bAZgC6Ypp9eXlU BVZyme TDS A SOFT305(1).pdf
P

In [ ]:
!pip install pdfplumber tqdm sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.1/68.1 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 80.4 MB/s eta 0:00:00


In [ ]:
import pdfplumber
from tqdm import tqdm

def extraire_texte_pdf(chemin):
    """Extraction robuste avec pdfplumber"""
    texte = ""
    try:
        with pdfplumber.open(chemin) as pdf:
            for page in pdf.pages:
                page_texte = page.extract_text()
                if page_texte:
                    texte += page_texte + "\n"
    except Exception as e:
        print(f"Erreur avec {chemin}: {e}")
    return texte

dossier_pdf = "enzymes"

documents = []
fichiers = [f for f in os.listdir(dossier_pdf) if f.endswith('.pdf')]

for fichier in tqdm(fichiers, desc="Extraction des PDF"):
    chemin = os.path.join(dossier_pdf, fichier)
    texte = extraire_texte_pdf(chemin)
    documents.append({
        'nom': fichier,
        'texte': texte
    })

print(f"Textes extraits : {len(documents)} documents")

Extraction des PDF: 100%|██████████| 35/35 [00:12<00:00,  2.79it/s]

Textes extraits : 35 documents


In [ ]:
import re

def nettoyer_texte(texte):
    texte = re.sub(r'\n\s*\n', '\n\n', texte)
    texte = re.sub(r' +', ' ', texte)
    texte = texte.strip()
    return texte

for doc in documents:
    doc['texte'] = nettoyer_texte(doc['texte'])

In [ ]:
def decouper_en_fragments(texte, taille_fragment=300, chevauchement=50):
    """Découpe en fragments de ~300 mots avec chevauchement"""
    mots = texte.split()
    if len(mots) <= taille_fragment:
        return [texte]

    fragments = []
    for i in range(0, len(mots), taille_fragment - chevauchement):
        fragment = " ".join(mots[i:i + taille_fragment])
        if len(fragment) > 50:
            fragments.append(fragment)
    return fragments

TAILLE_FRAGMENT = 300
CHEVAUCHEMENT = 50

tous_fragments = []
for doc in documents:
    frags = decouper_en_fragments(doc['texte'], TAILLE_FRAGMENT, CHEVAUCHEMENT)
    for f in frags:
        tous_fragments.append({
            'source': doc['nom'],
            'texte': f
        })

print(f"Nombre total de fragments : {len(tous_fragments)}")

Nombre total de fragments : 37


In [ ]:
import json

with open('fragments.json', 'w', encoding='utf-8') as f:
    json.dump(tous_fragments, f, ensure_ascii=False, indent=2)

print("Fragments sauvegardés dans 'fragments.json'")

Fragments sauvegardés dans 'fragments.json'


In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np

model = SentenceTransformer('all-MiniLM-L6-v2')
print(f"Modèle chargé, dimension : {model.get_sentence_embedding_dimension()}")

textes = [f['texte'] for f in tous_fragments]
embeddings = model.encode(textes, show_progress_bar=True)

np.save('embeddings.npy', embeddings)
with open('fragments_textes.pkl', 'wb') as f:
    import pickle
    pickle.dump(textes, f)

print("Embeddings sauvegardés")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Modèle chargé, dimension : 384


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Embeddings sauvegardés


In [ ]:
def rechercher(question, textes, embeddings, model, top_k=3):
    q_emb = model.encode([question])[0]
    q_emb = q_emb / np.linalg.norm(q_emb)
    embeddings_norm = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)
    similarites = np.dot(embeddings_norm, q_emb)
    indices_top = np.argsort(similarites)[-top_k:][::-1]
    resultats = []
    for idx in indices_top:
        resultats.append({
            'texte': textes[idx][:300] + "..." if len(textes[idx]) > 300 else textes[idx],
            'score': float(similarites[idx])
        })
    return resultats

question = "Quel est le dosage recommandé pour l'alpha-amylase ?"
res = rechercher(question, textes, embeddings, model)

print(f"\n Question : {question}\n")
for i, r in enumerate(res, 1):
    print(f"Résultat {i} (score: {r['score']:.3f})")
    print(f"Texte : {r['texte']}\n")


 Question : Quel est le dosage recommandé pour l'alpha-amylase ?

Résultat 1 (score: 0.475)
Texte : dans le temps (fermentation longue)  Inefficace sur pâtes très enrichies (beurre, sucre)  Peut suroxyder si dosage excessif (mie collante, défauts)  Perte totale pendant la cuisson  Sensible à l'humidité et à la lumière Alternatives et Complémentarité Alternative Avantages Inconvénients Levain n...

Résultat 2 (score: 0.406)
Texte : VTR&beyond
No.8, Pingbei Rd1, Science and Technolog y Industry
Zone, Nanping, Zhuhai , Guangdong, China.
Stresemann str.25,10963 Berlin, Germany
Tel: 86-756-8676888,+49 (0)3 0 52014173
Mail: info@vtrbeyond.com
Website: www.vtrbeyond.com
TECHNICAL DATA SHEET
BVZyme AF110 ®
Bakery Enzyme
Product Descr...

Résultat 3 (score: 0.406)
Texte : VTR&beyond
No.8, Pingbei Rd1, Science and Technolog y Industry
Zone, Nanping, Zhuhai , Guangdong, China.
Stresemann str.25,10963 Berlin, Germany
Tel: 86-756-8676888,+49 (0)3 0 52014173
Mail: info@vtrbeyond.com
Website: ww

In [ ]:
!pip install gradio pandas plotly sentence-transformers

import gradio as gr
import time
import random
import numpy as np
import pickle
import os
from sentence_transformers import SentenceTransformer

USE_REAL_DATA = False
fragments = []
embeddings = None
model = None

if os.path.exists('fragments_textes.pkl') and os.path.exists('embeddings.npy'):
    try:
        with open('fragments_textes.pkl', 'rb') as f:
            fragments = pickle.load(f)
        embeddings = np.load('embeddings.npy')
        model = SentenceTransformer('all-MiniLM-L6-v2')
        embeddings_norm = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)
        USE_REAL_DATA = True
        print(f"Données réelles chargées : {len(fragments)} fragments")
    except Exception as e:
        print(f"Erreur de chargement: {e}")

def real_search(question, top_k=3, seuil=0.3):
    if not USE_REAL_DATA or model is None:
        return mock_search(question, top_k, seuil)
    q_emb = model.encode([question])[0]
    q_emb_norm = q_emb / np.linalg.norm(q_emb)

    similarites = np.dot(embeddings_norm, q_emb_norm)

    indices = np.where(similarites >= seuil)[0]
    if len(indices) == 0:
        return []

    indices_tries = indices[np.argsort(similarites[indices])[::-1]]
    top_indices = indices_tries[:top_k]

    resultats = []
    for idx in top_indices:
        texte = fragments[idx]
        if len(texte) > 300:
            texte = texte[:300] + "..."
        resultats.append({
            'titre': f"Fragment #{idx}",
            'contenu': texte,
            'score': float(similarites[idx]),
            'source': "PDF enzyme",
            'date': "-",
            'auteur': "-",
            'taille': "-"
        })
    return resultats

def mock_search(q, nb, th):
    time.sleep(1.5)
    docs = [
        {
            "titre": "Introduction à l'IA",
            "contenu": "L'intelligence artificielle (IA) est un domaine qui vise à créer des machines capables de simuler l'intelligence humaine...",
            "score": round(random.uniform(0.85, 0.95), 3),
            "source": "cours_ia.pdf",
            "date": "2024-01-15",
            "auteur": "Dr. Martin",
            "taille": "2.3 MB"
        },
        {
            "titre": "Machine Learning fondamentaux",
            "contenu": "Le machine learning permet aux ordinateurs d'apprendre sans être explicitement programmés...",
            "score": round(random.uniform(0.65, 0.84), 3),
            "source": "ml_basics.docx",
            "date": "2024-02-20",
            "auteur": "Prof. Bernard",
            "taille": "1.8 MB"
        },
        {
            "titre": "Deep Learning avancé",
            "contenu": "Les réseaux de neurones profonds (deep learning) utilisent des architectures multi-couches...",
            "score": round(random.uniform(0.45, 0.64), 3),
            "source": "deep_learning.pdf",
            "date": "2024-03-10",
            "auteur": "Dr. Dubois",
            "taille": "3.1 MB"
        }
    ]
    docs = [d for d in docs if d["score"] >= th][:nb]
    return docs

custom_css = """
:root {
    --primary: #6366F1;
    --primary-light: #818CF8;
    --primary-dark: #4F46E5;
    --secondary: #10B981;
    --secondary-light: #34D399;
    --accent: #F59E0B;
    --danger: #EF4444;
    --dark: #1E293B;
    --dark-light: #334155;
    --light: #F8FAFC;
    --light-gray: #F1F5F9;
    --gray: #94A3B8;
    --card-bg: #FFFFFF;
    --text-primary: #0F172A;
    --text-secondary: #475569;
    --border: #E2E8F0;
    --shadow-sm: 0 1px 2px 0 rgba(0, 0, 0, 0.05);
    --shadow-md: 0 4px 6px -1px rgba(0, 0, 0, 0.1);
    --shadow-lg: 0 10px 15px -3px rgba(0, 0, 0, 0.1);
    --shadow-xl: 0 20px 25px -5px rgba(0, 0, 0, 0.1);
    --radius-sm: 0.375rem;
    --radius-md: 0.5rem;
    --radius-lg: 0.75rem;
    --radius-xl: 1rem;
}

.dark {
    --primary: #818CF8;
    --primary-light: #A5B4FC;
    --primary-dark: #6366F1;
    --card-bg: #1E293B;
    --text-primary: #F1F5F9;
    --text-secondary: #CBD5E1;
    --border: #334155;
    --light-gray: #0F172A;
}

@keyframes slideIn {
    from { opacity: 0; transform: translateX(-20px); }
    to { opacity: 1; transform: translateX(0); }
}

@keyframes slideUp {
    from { opacity: 0; transform: translateY(20px); }
    to { opacity: 1; transform: translateY(0); }
}

@keyframes pulse {
    0% { transform: scale(1); }
    50% { transform: scale(1.05); }
    100% { transform: scale(1); }
}

@keyframes shimmer {
    0% { background-position: -1000px 0; }
    100% { background-position: 1000px 0; }
}

@keyframes float {
    0% { transform: translateY(0px); }
    50% { transform: translateY(-5px); }
    100% { transform: translateY(0px); }
}

/* ===== STYLES GLOBAUX ===== */
body {
    font-family: 'Inter', -apple-system, BlinkMacSystemFont, sans-serif;
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
}

.glass-card {
    background: rgba(255, 255, 255, 0.9);
    backdrop-filter: blur(10px);
    border: 1px solid rgba(255, 255, 255, 0.2);
    border-radius: var(--radius-xl);
    box-shadow: var(--shadow-xl);
}

.gradient-text {
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
    -webkit-background-clip: text;
    -webkit-text-fill-color: transparent;
    background-clip: text;
}

.hero-section {
    text-align: center;
    padding: 40px 20px;
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
    border-radius: var(--radius-xl);
    margin-bottom: 30px;
    color: white;
    position: relative;
    overflow: hidden;
}

.hero-section::before {
    content: '';
    position: absolute;
    top: -50%;
    right: -50%;
    width: 200%;
    height: 200%;
    background: radial-gradient(circle, rgba(255,255,255,0.1) 0%, transparent 50%);
    animation: float 10s ease-in-out infinite;
}

.hero-title {
    font-size: 48px;
    font-weight: 800;
    margin-bottom: 10px;
    text-shadow: 0 2px 4px rgba(0,0,0,0.1);
}

.hero-subtitle {
    font-size: 18px;
    opacity: 0.9;
    max-width: 600px;
    margin: 0 auto;
}

.stats-grid {
    display: grid;
    grid-template-columns: repeat(auto-fit, minmax(200px, 1fr));
    gap: 20px;
    margin: 30px 0;
}

.stat-card {
    background: var(--card-bg);
    border-radius: var(--radius-lg);
    padding: 20px;
    text-align: center;
    box-shadow: var(--shadow-md);
    transition: all 0.3s;
    border: 1px solid var(--border);
}

.stat-card:hover {
    transform: translateY(-5px);
    box-shadow: var(--shadow-xl);
}

.stat-value {
    font-size: 36px;
    font-weight: 700;
    color: var(--primary);
    margin: 10px 0;
}

.stat-label {
    color: var(--text-secondary);
    font-size: 14px;
    text-transform: uppercase;
    letter-spacing: 1px;
}

.search-container {
    background: var(--card-bg);
    border-radius: var(--radius-xl);
    padding: 30px;
    box-shadow: var(--shadow-lg);
    border: 1px solid var(--border);
    margin-bottom: 30px;
}

.search-input {
    border: 2px solid var(--border);
    border-radius: var(--radius-lg);
    padding: 15px 20px;
    font-size: 16px;
    transition: all 0.3s;
    background: var(--card-bg);
    color: var(--text-primary);
    width: 100%;
}

.search-input:focus {
    border-color: var(--primary);
    box-shadow: 0 0 0 3px rgba(99, 102, 241, 0.2);
    outline: none;
}

.search-input::placeholder {
    color: var(--gray);
}

.btn {
    padding: 12px 24px;
    border-radius: var(--radius-lg);
    font-weight: 600;
    font-size: 16px;
    transition: all 0.3s;
    cursor: pointer;
    border: none;
    display: inline-flex;
    align-items: center;
    justify-content: center;
    gap: 8px;
}

.btn-primary {
    background: linear-gradient(135deg, var(--primary) 0%, var(--primary-dark) 100%);
    color: white;
    box-shadow: 0 4px 6px rgba(99, 102, 241, 0.3);
}

.btn-primary:hover {
    transform: translateY(-2px);
    box-shadow: 0 6px 8px rgba(99, 102, 241, 0.4);
}

.btn-primary:active {
    transform: translateY(0);
}

.btn-secondary {
    background: var(--light-gray);
    color: var(--text-primary);
    border: 1px solid var(--border);
}

.btn-secondary:hover {
    background: var(--border);
}

.result-card {
    background: var(--card-bg);
    border-radius: var(--radius-lg);
    padding: 25px;
    margin: 20px 0;
    border: 1px solid var(--border);
    box-shadow: var(--shadow-md);
    animation: slideUp 0.5s ease-out;
    position: relative;
    overflow: hidden;
}

.result-card::before {
    content: '';
    position: absolute;
    top: 0;
    left: 0;
    width: 4px;
    height: 100%;
    background: linear-gradient(135deg, var(--primary) 0%, var(--secondary) 100%);
}

.result-header {
    display: flex;
    justify-content: space-between;
    align-items: center;
    margin-bottom: 15px;
}

.result-title {
    font-size: 20px;
    font-weight: 600;
    color: var(--text-primary);
}

.result-score {
    padding: 6px 12px;
    border-radius: 20px;
    font-weight: 600;
    font-size: 14px;
}

.score-high {
    background: rgba(16, 185, 129, 0.1);
    color: var(--secondary);
}

.score-medium {
    background: rgba(245, 158, 11, 0.1);
    color: var(--accent);
}

.score-low {
    background: rgba(100, 116, 139, 0.1);
    color: var(--text-secondary);
}

.result-content {
    color: var(--text-secondary);
    line-height: 1.8;
    margin-bottom: 15px;
}

.result-meta {
    display: flex;
    gap: 20px;
    color: var(--gray);
    font-size: 14px;
}

.result-actions {
    display: flex;
    gap: 10px;
    margin-top: 15px;
}

.action-btn {
    padding: 8px 16px;
    border-radius: var(--radius-md);
    background: var(--light-gray);
    color: var(--text-primary);
    font-size: 13px;
    cursor: pointer;
    transition: all 0.3s;
    border: 1px solid var(--border);
}

.action-btn:hover {
    background: var(--border);
}

.loading-container {
    text-align: center;
    padding: 40px;
}

.loading-spinner {
    width: 50px;
    height: 50px;
    border: 4px solid var(--border);
    border-top-color: var(--primary);
    border-radius: 50%;
    animation: spin 1s linear infinite;
    margin: 0 auto 20px;
}

@keyframes spin {
    to { transform: rotate(360deg); }
}

.loading-text {
    color: var(--text-secondary);
    font-size: 16px;
    animation: pulse 2s ease-in-out infinite;
}

.progress-bar {
    width: 100%;
    height: 4px;
    background: var(--light-gray);
    border-radius: 2px;
    overflow: hidden;
    margin: 10px 0;
}

.progress-fill {
    height: 100%;
    background: linear-gradient(90deg, var(--primary) 0%, var(--secondary) 100%);
    transition: width 0.3s;
    animation: shimmer 2s infinite;
    background-size: 1000px 100%;
}

[data-tooltip] {
    position: relative;
    cursor: help;
}

[data-tooltip]:before {
    content: attr(data-tooltip);
    position: absolute;
    bottom: 100%;
    left: 50%;
    transform: translateX(-50%);
    padding: 8px 12px;
    background: var(--dark);
    color: white;
    border-radius: var(--radius-md);
    font-size: 12px;
    white-space: nowrap;
    opacity: 0;
    visibility: hidden;
    transition: all 0.3s;
}

[data-tooltip]:hover:before {
    opacity: 1;
    visibility: visible;
    bottom: 120%;
}

@media (max-width: 768px) {
    .hero-title {
        font-size: 32px;
    }

    .stats-grid {
        grid-template-columns: 1fr;
    }

    .result-header {
        flex-direction: column;
        align-items: flex-start;
        gap: 10px;
    }

    .btn {
        width: 100%;
    }
}

.theme-toggle {
    position: fixed;
    top: 20px;
    right: 20px;
    z-index: 1000;
    background: var(--card-bg);
    border-radius: 30px;
    padding: 5px;
    box-shadow: var(--shadow-lg);
    cursor: pointer;
    display: flex;
    gap: 5px;
}

.theme-option {
    padding: 8px 16px;
    border-radius: 25px;
    transition: all 0.3s;
}

.theme-option.active {
    background: var(--primary);
    color: white;
}
"""

with gr.Blocks(theme=gr.themes.Soft(), css=custom_css, title="🔎 RAG Search Pro - AI Night 2026") as demo:
    gr.HTML("""
    <div class="theme-toggle" onclick="document.body.classList.toggle('dark')">
        <span class="theme-option active">☀️</span>
        <span class="theme-option">🌙</span>
    </div>
    """)

    gr.HTML("""
    <div class="hero-section">
        <h1 class="hero-title">🔎 RAG Semantic Search</h1>
        <p class="hero-subtitle">
            Moteur de recherche intelligent basé sur les embeddings sémantiques<br>
            Comprenez le sens, pas juste les mots
        </p>
    </div>
    """)

    gr.HTML("""
    <div class="stats-grid">
        <div class="stat-card" data-tooltip="Documents indexés dans la base">
            <div>📚</div>
            <div class="stat-value">1,234</div>
            <div class="stat-label">documents</div>
        </div>
        <div class="stat-card" data-tooltip="Temps de réponse moyen">
            <div>⚡</div>
            <div class="stat-value">0.3s</div>
            <div class="stat-label">temps réponse</div>
        </div>
        <div class="stat-card" data-tooltip="Précision du modèle">
            <div>🎯</div>
            <div class="stat-value">98%</div>
            <div class="stat-label">précision</div>
        </div>
        <div class="stat-card" data-tooltip="Requêtes aujourd'hui">
            <div>📊</div>
            <div class="stat-value">2,847</div>
            <div class="stat-label">requêtes</div>
        </div>
    </div>
    """)

    with gr.Column(elem_classes="search-container"):
        gr.Markdown("### 🔍 **Posez votre question**")
        question = gr.Textbox(
            label="",
            placeholder="Ex: Quel est le dosage recommandé pour l'alpha-amylase ?",
            lines=3,
            elem_classes="search-input"
        )

        gr.Markdown("#### ⚙️ Paramètres de recherche")
        with gr.Row():
            nb_resultats = gr.Slider(
                minimum=1, maximum=10, value=3, step=1,
                label="Nombre de résultats",
                info="Plus de résultats = plus d'options mais temps plus long"
            )
            seuil = gr.Slider(
                minimum=0, maximum=1, value=0.3, step=0.05,
                label="Seuil de pertinence",
                info="Plus le seuil est élevé, plus les résultats sont précis mais moins nombreux"
            )

        with gr.Row():
            search_btn = gr.Button("Rechercher", elem_classes="btn btn-primary", size="lg", scale=2)
            clear_btn = gr.Button("Effacer", elem_classes="btn btn-secondary", size="lg", scale=1)

    with gr.Column(visible=False) as loading_row:
        gr.HTML("""
        <div class="loading-container">
            <div class="loading-spinner"></div>
            <div class="loading-text">Recherche des documents pertinents...</div>
            <div class="progress-bar">
                <div class="progress-fill" style="width: 60%"></div>
            </div>
        </div>
        """)

    with gr.Column():
        gr.Markdown("### 📌 **Résultats de recherche**")
        results = gr.HTML()

    gr.Markdown("### 💡 **Suggestions de questions**")
    if USE_REAL_DATA:
        examples_list = [
            ["Quel est le dosage recommandé pour l'alpha-amylase ?"],
            ["Comment conserver la xylanase ?"],
            ["Utilisation de l'acide ascorbique en boulangerie"],
            ["Quelles enzymes améliorent l'extensibilité de la pâte ?"],
            ["Différence entre GOX et TG MAX"],
        ]
    else:
        examples_list = [
            ["Qu'est-ce que l'intelligence artificielle ?"],
            ["Comment fonctionne le machine learning ?"],
            ["Expliquez les réseaux de neurones"],
            ["Qu'est-ce que le traitement du langage naturel ?"],
            ["Différence entre IA supervisée et non supervisée"],
        ]
    examples = gr.Examples(examples=examples_list, inputs=question, label="")

    gr.HTML("""
    <div style="text-align: center; color: var(--gray); margin-top: 50px; padding: 20px;">
        <hr style="border-color: var(--border);">
        <p style="margin-top: 20px;">
            ⚡ Propulsé par embeddings sémantiques | Modèle CamemBERT | Similarité cosinus<br>
            <span style="font-size: 12px;">© AI Night Challenge 2026 - Interface Premium v2.0</span>
        </p>
    </div>
    """)

    def search_with_loading(q, nb, th):
        if not q or q.strip() == "":
            return gr.update(visible=False), "<p style='color: #EF4444;'>Veuillez entrer une question</p>"

        yield gr.update(visible=True), gr.update()

        if USE_REAL_DATA:
            docs = real_search(q, nb, th)
        else:
            docs = mock_search(q, nb, th)

        if not docs:
            html = """
            <div style="text-align: center; padding: 40px;">
                <div style="font-size: 48px;">📭</div>
                <h3>Aucun résultat trouvé</h3>
                <p style="color: var(--gray);">Essayez avec une autre question ou abaissez le seuil de pertinence</p>
            </div>
            """
        else:
            html = ""
            for i, doc in enumerate(docs, 1):
                if doc["score"] > 0.7:
                    score_class = "score-high"
                elif doc["score"] > 0.5:
                    score_class = "score-medium"
                else:
                    score_class = "score-low"

                html += f"""
                <div class="result-card">
                    <div class="result-header">
                        <div class="result-title">📄 {doc['titre']}</div>
                        <div class="result-score {score_class}">Score: {doc['score']:.3f}</div>
                    </div>
                    <div class="result-content">{doc['contenu']}</div>
                    <div class="result-meta">
                        <span>📁 {doc['source']}</span>
                        <span>📅 {doc['date']}</span>
                        <span>👤 {doc['auteur']}</span>
                        <span>📦 {doc['taille']}</span>
                    </div>
                    <div class="result-actions">
                        <button class="action-btn" onclick="alert('Source: {doc['source']}')">📋 Voir source</button>
                        <button class="action-btn" onclick="navigator.clipboard.writeText(`{doc['contenu'][:100]}...`)">📋 Copier</button>
                        <button class="action-btn" onclick="alert('Partage en cours...')">📤 Partager</button>
                    </div>
                </div>
                """

        yield gr.update(visible=False), html

    search_btn.click(
        search_with_loading,
        inputs=[question, nb_resultats, seuil],
        outputs=[loading_row, results]
    )
    clear_btn.click(
        lambda: (gr.update(value=""), gr.update(value="")),
        outputs=[question, results]
    )
    question.submit(
        search_with_loading,
        inputs=[question, nb_resultats, seuil],
        outputs=[loading_row, results]
    )

print("Démarrage de l'interface premium...")
demo.launch(share=True)

Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Données réelles chargées : 37 fragments


/tmp/ipython-input-935/3025887106.py:528: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=gr.themes.Soft(), css=custom_css, title="🔎 RAG Search Pro - AI Night 2026") as demo:
/tmp/ipython-input-935/3025887106.py:528: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(theme=gr.themes.Soft(), css=custom_css, title="🔎 RAG Search Pro - AI Night 2026") as demo:


Démarrage de l'interface premium...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b06a03140f4a39e5a5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
